# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm 
# import math
from functools import partial

In [ ]:
DS_name    = 'CelebA'
model_type = 'pretrained'

# Path Setting
**NOTE** Comment uncomment LINE No 4,5 to change dataset path

In [ ]:
path_curr = os.getcwd()
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

#path_dataset = os.path.abspath(os.path.join(path_notebook,'dataset/CelebA/CelebAMask-HQ'))
path_dataset = 'D:\\DS\\CelebAMask-HQ\\'

path_main = path_curr

results_path_main   = os.path.join(path_curr,               'results')
# path_csv            = os.path.join(results_path_main,           'csv')
path_csv    = '../results_logits_with_auc_clipped/'
utils_path          = os.path.join(path_notebook,             'utils')

print('path_dataset: \t',        path_dataset)
print('codes path: \t',             path_curr)
print('csv path: \t',                path_csv)
print('utils path: \t',            utils_path)

In [ ]:
csv_anno_file = f'{path_dataset}//CelebA-HQ-attribute.txt'
df_attr = pd.read_csv(csv_anno_file, delim_whitespace=True,skiprows=[0])
df_attr.replace(-1,0,inplace=True)
df_attr.head(2)
print(len(np.unique(df_attr.image_id)))

classes_names = []
for i,x in  enumerate(df_attr.columns):
    # print(i-1, x)
    if i!=0:
        classes_names.append(x)
#df_attr_sel = df_attr[['image_id','Brown_Hair','Eyeglasses']].copy()
df_attr_sel = df_attr[['image_id','Brown_Hair','Black_Hair','Blond_Hair','Gray_Hair','Eyeglasses']].copy()

#df_attr_sel = df_attr_sel[(df_attr_sel.Brown_Hair==1) & (df_attr_sel.Eyeglasses==1) ]
df_attr_sel = df_attr_sel[(df_attr_sel.Brown_Hair==1) | (df_attr_sel.Black_Hair==1) |(df_attr_sel.Blond_Hair==1) |(df_attr_sel.Gray_Hair==1) |(df_attr_sel.Eyeglasses==1) ]

df_attr_sel.head()

files = np.unique(df_attr_sel.image_id)
files_sub = files[:200]
iterations = len(files_sub)
iterations

# Setting for CSV File Name

In [ ]:
# background_type        = 'blurred'     # black , white , gray , noise , blurred , full
# background_type        = 'black'     # black , white , gray , noise , blurred , full
# background_type        = 'white'     # black , white , gray , noise , blurred , full
# background_type        = 'noise'     # black , white , gray , noise , blurred , full
background_type        = 'gray'     # black , white , gray , noise , blurred , full
# background_type        = 'blurred'     # black , white , gray , noise , blurred , full

In [ ]:
bg_type = ('bg_'+background_type)

results_path      = os.path.join(results_path_main,bg_type)

In [ ]:
def get_bpt_heatmaps(num_samples=None,verbose=None):
    pass
def get_aa_heatmaps(num_samples=None,verbose=None):
    pass
def get_lime_heatmaps(num_samples=None,num_segments=None,verbose=None):
    pass
def get_LRP_captum_heatmaps():
    pass
def get_gradcam_heatmaps():
    pass
def methods_gradcam():
    pass
def get_idg_heatmaps(use_abs=True):
    pass
def get_gradexpl_heatmap():
    pass
def get_gradshap_captum_heatmaps():
    pass

In [ ]:
#     name,                 color,                  functor
verbose = False
methods = [ 
    ('BPT-100',         'xkcd:light pink',     partial(get_bpt_heatmaps, num_samples=100,verbose=verbose)),
    ('BPT-500',         'xkcd:bright pink',     partial(get_bpt_heatmaps, num_samples=500,verbose=verbose)),
    ('BPT-1000',         'xkcd:deep pink',     partial(get_bpt_heatmaps, num_samples=1000,verbose=verbose)),
    ]

methods_aa = [ # if multiple backgrounds
    ('AA-100', 'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=100, verbose=verbose)),
    ('AA-500', 'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=500, verbose=verbose)),
    ('AA-1000', 'xkcd:bright blue',    partial(get_aa_heatmaps, num_samples=1000, verbose=verbose)),
]
methods_lime = [
    ('LIME-250',        'xkcd:bright lime',     partial(get_lime_heatmaps, num_segments=50, num_samples=250,verbose=verbose)),
    ('LIME-500',        'xkcd:kermit green',   partial(get_lime_heatmaps, num_segments=100, num_samples=500,verbose=verbose)),
    ('LIME-1000',        'xkcd:dark lime green',partial(get_lime_heatmaps, num_segments=200, num_samples=1000,verbose=verbose))
    ]

methods_LRP = [
    ('LRP',        'xkcd:bright lime',     get_LRP_captum_heatmaps)
    ]
methods_gradcam = [
    ('GradCAM',     'xkcd:camel',            get_gradcam_heatmaps)
    ]
methods_cam = [
    ('aIDG',         'xkcd:indigo',          partial(get_idg_heatmaps, use_abs=False)),
    ('aGradExpl',    'red',                  partial(get_gradexpl_heatmap, use_abs=False))
    ]

methods_GradShapE = [
    ('GradShapE',     'xkcd:camel',            partial(get_gradshap_captum_heatmaps, n_samples=50, use_abs=False))
]

methods += methods_aa
methods += methods_lime
methods += methods_LRP
methods += methods_gradcam
methods += methods_cam
methods += methods_GradShapE

for n,_,_ in methods:
    print(n)

In [ ]:
exp_type = path_curr.split('\\')[-1].split('_')[0]
print(exp_type)
csv_filename = f'{path_csv}/csv_expIoU_face_{iterations}_{len(methods)}_{background_type}.csv'
print(csv_filename,os.path.exists(csv_filename))

In [ ]:
df_main= pd.read_csv(csv_filename)

In [ ]:
df_main.head()

In [ ]:
# auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_mse', 'aucD_mse','time_exp']
auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_mse', 'aucD_mse','max_IoU','au_IoU','time_exp']

# CSS Files

## Create CSS Files

In [ ]:
# GENERATE_FILE = True
# if GENERATE_FILE:
#     with open(f'{utils_path}/html_stylesheet.txt', 'w') as f:
#         f.write("""
#                 <style>
#                     table.space {margin-top: 10px;padding-top: 10px;width: 100%;} 
#                     table{border: 1px solid black;},
#                     th{font-size: 2em},tr{font-size: 2em},
#                     .collapsible-title {
#                       font-size: 18px;
#                       font-weight: bold;
#                       color: #333;
#                     }
#                     .collapsible-text {
#                       font-size: 16px;
#                       color: #666;
#                       line-height: 1.5;
#                     }
#                     .collapsible-content {
#                       font-size: 20px;}
#                     table {
#                       text-align: left;
#                       position: relative;
#                       border-collapse: separate; 
#                     border-top: 2px solid;
#                     border-bottom: 2px solid;
#                     border-right: 2px solid;
#                     }
#                     td {
#                     padding: 0.1rem;
#                     }
#                     th {
#                         border: 2px solid black;
#                         padding: 0.1rem;
#                     }
#                     tr.red th {
#                       background: red;
#                       color: white;
#                     }
#                     tr.green th {
#                       background: green;
#                       color: white;
#                     }
#                     tr.purple th {
#                       background: purple;
#                       color: white;
#                     }
#                     th {
#                     border-radius: 3px;
#                     background-color: rgba(255, 255, 255, 0.8) !important;
#                     text-align:center;
#                     background: white;
#                     position: sticky;
#                     top: 0;
#                     box-shadow: 0 2px 2px -1px rgba(0, 0, 0, 0.4);
#                     }
#                     tr:hover {background-color: rgba(234, 242, 215,0.8);}
#                     .switch {
#                     position: relative;
#                     display: inline-block;
#                     width: 60px;
#                     height: 34px;
#                     }
#                     .switch input {
#                     opacity: 0;
#                     width: 0;
#                     height: 0;
#                     }
#                     .slider {
#                     position: absolute;
#                     cursor: pointer;
#                     top: 0;
#                     left: 0;
#                     right: 0;
#                     bottom: 0;
#                     background-color: #808080;
#                     -webkit-transition: .4s;
#                     transition: .4s;
#                     }
#                     .slider:before {
#                     position: absolute;
#                     content: "";
#                     height: 26px;
#                     width: 26px;
#                     left: 4px;
#                     bottom: 4px;
#                     background-color: white;
#                     -webkit-transition: .4s;
#                     transition: .4s;
#                     }
#                     input:checked + .slider {
#                     background-color: #2196F3;
#                     }
#                     input:focus + .slider {
#                     box-shadow: 0 0 1px #2196F3;
#                     }
#                     input:checked + .slider:before {
#                     -webkit-transform: translateX(26px);
#                     -ms-transform: translateX(26px);
#                     transform: translateX(26px);
#                     }
#                     .slider.round {
#                     border-radius: 34px;
#                     }
#                     .slider.round:before {
#                     border-radius: 50%;
#                     }
#                     #topBtn {
#                         position: fixed;
#                         bottom: 5px;
#                         float: right;
#                         right: 18.5%;
#                         left: 0.25%;
#                         max-width: 70px;
#                         width: 100%;
#                         font-size: 18px;
#                         border-color: rgba(85, 85, 85, 0.8);
#                         background-color: rgba(27, 163, 156,0.9);
#                         padding: 2px;
#                         border-radius: 10px;
#                         }
#                     #topBtn:hover {
#                         background-color: #7dbbf1;
#                     }
#                     table {border: none;}
#                     * {
#                     box-sizing: border-box;
#                     }
#                     .box {
#                     float: left;
#                     width: 100%;
#                     padding: 10px;
#                     }
#                     .clearfix::after {
#                     content: "";
#                     clear: both;
#                     display: table;
#                     }
#                     table {
#                     height: 100%;
#                     border-collapse: seperate;
#                     width: 100%;
#                     margin: 10px;
#                     font-size: 0.8em;
#                     }
#                     thead {
#                     border-radius: 3px;
#                     position: sticky;
#                     top: 0;
#                     background: #eee;
#                     }
#                     header h1 {
#                         font-size: 70px;
#                         font-weight: 600;
#                         background-image: linear-gradient(to left, #553c9a, #b393d3);
#                         color: transparent;
#                         background-clip: text;
#                         -webkit-background-clip: text;
#                         text-shadow: 0px 4px 10px rgba(179, 147, 211, 0.8)
#                     }
#                     h1 {
#                       font-size: 30px;
#                       line-height: 1.8;
#                       text-transform: uppercase;
#                       font-family: "Montserrat", sans-serif;
#                     }
#                 .hero {
#                     position: relative;
#                     background: #333 url(http://srdjanpajdic.com/slike/2.jpg) no-repeat center center fixed;
#                     -webkit-background-size: cover;
#                     -moz-background-size: cover;
#                     background-size: cover;
#                     text-align: center;
#                     color: #fff;
#                     padding-top: 20px;
#                     min-height: 300px;
#                     letter-spacing: 2px;
#                     font-family: "Montserrat", sans-serif;
#                     h1 {
#                     font-size: 50px;
#                     line-height: 1.3;
#                     span {
#                     font-size: 25px;
#                     color: $color2;
#                     border-bottom: 2px solid $color2;
#                     padding-bottom: 8px;
#                     line-height: 3;
#                     }
#                 }
#                 .main_container{
#                    width:99%;
#                 }
#                 html, body {
#                 height:100%;
#                 width:99%;
#                 margin:0;
#                 padding:0;
#                 }
#                     </style>
#                     """)
#     with open(f'{utils_path}/html_head.txt', 'w') as f:
#         f.write("""
#             <head>
#                 <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css"
#                     rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" 
#                     crossorigin="anonymous">
#                 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js"
#                     integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous">
#                 </script>
#             </head>
#             """)
#     with open(f'{utils_path}/html_footer_scripts_full.txt', 'w') as f:
#         f.write("""
#             <script>const toggleAllButton = document.getElementById('toggleAllButton');
#                 const collapsibleContents = document.querySelectorAll('.collapsible-content');
#                 toggleAllButton.addEventListener('click', () => {
#                   collapsibleContents.forEach(content => {
#                     if (content.style.display === 'none') {
#                       content.style.display = 'block';
#                       content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
#                     } else {
#                       content.style.display = 'none';
#                       content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
#                     }
#                   });
#                 });
#             </script>
#             """)
#     with open(f'{utils_path}/html_footer_scripts_IoU.txt', 'w') as f:
#         f.write("""<script>
#         const toggleAllButton = document.getElementById('toggleAllButton');
#             const collapsibleContents = document.querySelectorAll('.collapsible-content');
#             toggleAllButton.addEventListener('click', () => {
#               collapsibleContents.forEach(content => {
#               document.getElementById('data_rows').style.border = 'none';
#                 if (content.style.display === 'none') {
#                   content.style.display = 'block';
#                   content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
#                 } else {
#                   content.style.display = 'none';
#                   content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
#                 }
#               });
#             });
#             </script>
#            """)

## Load CSS Files

In [ ]:
with open(f"{utils_path}/html_stylesheet.txt", "r+") as f:
    html_stylesheet = f.read()
with open(f"{utils_path}/html_footer_scripts_IoU.txt", "r+") as f:
    html_footer_scripts_IoU = f.read()
with open(f"{utils_path}/html_footer_scripts_full.txt", "r+") as f:
    html_footer_scripts_full = f.read()
with open(f"{utils_path}/html_head.txt", "r+") as f:
    html_head = f.read()

In [ ]:
files = list(df_main.image)
files = np.unique(files)

In [ ]:
df_main.head(3)

In [ ]:
df_main.intrested_class_lbl.unique()

In [ ]:
df_main = df_main[df_main.intrested_class_lbl != 'Gray_Hair']

df_main.intrested_class_lbl.unique()

In [ ]:
explained_classes_id = np.unique(df_main.intrested_class)
explained_classes = [classes_names[x] for x in explained_classes_id]
# explained_classes = [(x,classes_names[x]) for x in explained_classes_id]
# explained_classes

intrested_class = 'Brown_Hair'

In [ ]:
# for ec_id,ec in explained_classes:
#     if ec == 'Brown_Hair':
#         intrested_class_id = ec_id
# print(intrested_class_id,intrested_class)

## Filter Dataframe for Selected Class

In [ ]:
filter_df = False

if filter_df:
    df = df_main[df_main['intrested_class_lbl']==intrested_class]
else:
    df = df_main.copy()
print(len(df))


# Reduce File Size

In [ ]:
import os
from PIL import Image

# Define input and output directories
input_folder = f'results/{bg_type}'
output_folder = f"results/{bg_type}_red"
new_dpi = (72, 72)  # Set new DPI

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process all images in the folder
for filename in tqdm(os.listdir(input_folder)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):

        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Open the image
        with Image.open(input_path) as img:
            # Save with new DPI
            if filename.split('.')[0].split('_')[-1]=='iou':
                print
                img.save(output_path)#, dpi=new_dpi)
            else:
                new_size = (img.width // 2, img.height // 2)
                img = img.resize(new_size, Image.LANCZOS)

                img.save(output_path)#, dpi=new_dpi)
            del img
        # print(f"Processed: {filename}")

print("✅ All images processed and saved with reduced DPI!")


# HTML FILES

In [ ]:
file_success = []

for fff_id,fff in enumerate(tqdm(files)):
    for ecc_id,ecc in zip(explained_classes_id,explained_classes):
        img_path = f'{results_path}//{fff}_{ecc}_{background_type}_heatmaps.png'
        # print(img_path)
        if os.path.exists(img_path):
            file_success.append(img_path)
print(len(file_success))

# file_success[0]

# Use Subset of FILES

In [ ]:
use_subset = True

if use_subset:
    subset_count = 100

In [ ]:
if use_subset:
    files_set = file_success[:subset_count]
else:
    files_set = file_success

In [ ]:
files_set

In [ ]:
results_path      = os.path.join(results_path_main,f'{bg_type}_red')
results_path

## HTML FILE FOR HEATMAPS

In [ ]:
nf_images = []
print(f'Total Images: {len(files)}\t Total Methods {len(methods)}')
html_f_name = f'{results_path_main}/HTML_{exp_type}_{DS_name}_{background_type}.html'

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_type}, background_type: {background_type}] - No of Images:{len(files_set)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth,_,_ in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')
        
for fff_id,fff in enumerate(tqdm(files_set)):
    file_n = fff
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    
    PP = df[df.image==file_n].f_S.iloc[0]
    PL = df[df.image==file_n].f_S

    for ecc_id,ecc in zip(explained_classes_id,explained_classes):
        
        img_path = f'{results_path}//{file_n}_{ecc}_{background_type}_heatmaps.png'
        if os.path.exists(img_path):
            with open(html_f_name, 'a') as f:
                f.write(f'<tr><td colspan="{len(methods)+2}">\
                        <img src={img_path} width="99.9%"> <br>\
                            <p>{file_n}  - {ecc} -  {file_n}  -  {PP:0.5}  -  \
                            </p>\
                        </td>\
                        ')
            with open(html_f_name, 'a') as f:
                    f.write(f'\
                        <tr><td">\
                          ')
            df11 = df[df.intrested_class==ecc_id]
            for sm in auc_methods:
                df1 = df11[(df11.image==file_n)].eval(f'{sm}')
                with open(html_f_name, 'a') as f:
                    f.write(f'\
                            <tr><td>\
                            <div class="collapsible-container">\
                              <div class="collapsible-header">\
                                <span class="toggle-icon"></span>\
                              </div>\
                              <div class="collapsible-content">\
                                <p>\
                                <b>{sm}</b>\
                                </div>\
                            </div>\
                            </td>\
                            ')
                for meth_id,meth in enumerate(methods):
                    df2 = df1.iloc[meth_id]
                    with open(html_f_name, 'a') as f:
                        f.write(f'\
                                <td> \
                                <div class="collapsible-container">\
                                  <div class="collapsible-header">\
                                    <span class="toggle-icon"></span>\
                                  </div>\
                                  <div class="collapsible-content">\
                                    <p>\
                                    {df2:0.5} </p>\
                                    </div>\
                                </div>\
                                </td>\
                                ')
                with open(html_f_name, 'a') as f:
                    f.write(f'\
                    </tr>\
                    ')
        with open(html_f_name, 'a') as f:
            f.write('''
                        </div>
                    </div>
                    </td></tr>
                    ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

## HTML FILE FOR IOU ONLY

In [ ]:
nf_images = []
print(f'Total Images: {len(files)}\t Total Methods {len(methods)}')
html_f_name = f'{results_path_main}/HTML_{exp_type}_{DS_name}_{background_type}_IoU.html'

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_type}, background_type: {background_type}] - No of Images:{len(file_success)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth,_,_ in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')
        
for fff_id,fff in enumerate(tqdm(files)):
    file_n = fff
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    PP = df[df.image==file_n].f_S.iloc[0]
    PL = df[df.image==file_n].f_S

    for ecc_id,ecc in zip(explained_classes_id,explained_classes):
        
        img_path = f'{results_path}//{file_n}_{ecc}_{background_type}_iou.png'
        if os.path.exists(img_path):
            with open(html_f_name, 'a') as f:
                f.write(f'<tr><td colspan="{len(methods)+2}">\
                        <img src={img_path} width="99.9%"> <br>\
                            <p>{file_n}  - {ecc} -  {file_n}  -  {PP:0.5}  -  \
                            </p>\
                        </td>\
                        ')
            with open(html_f_name, 'a') as f:
                    f.write(f'\
                        <tr><td">\
                          ')
            df11 = df[df.intrested_class==ecc_id]
            for sm in auc_methods:
                df1 = df11[(df11.image==file_n)].eval(f'{sm}')
                with open(html_f_name, 'a') as f:
                    f.write(f'\
                            <tr><td>\
                            <div class="collapsible-container">\
                              <div class="collapsible-header">\
                                <span class="toggle-icon"></span>\
                              </div>\
                              <div class="collapsible-content">\
                                <p>\
                                <b>{sm}</b>\
                                </div>\
                            </div>\
                            </td>\
                            ')
                for meth_id,meth in enumerate(methods):
                    df2 = df1.iloc[meth_id]
                    with open(html_f_name, 'a') as f:
                        f.write(f'\
                                <td> \
                                <div class="collapsible-container">\
                                  <div class="collapsible-header">\
                                    <span class="toggle-icon"></span>\
                                  </div>\
                                  <div class="collapsible-content">\
                                    <p>\
                                    {df2:0.5} </p>\
                                    </div>\
                                </div>\
                                </td>\
                                ')
            with open(html_f_name, 'a') as f:
                f.write(f'\
                </tr>\
                ')
        with open(html_f_name, 'a') as f:
            f.write('''
                        </div>
                    </div>
                    </td></tr>
                    ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

## Combined HTML File

In [ ]:
nf_images = []
print(f'Total Images: {len(files)}\t Total Methods {len(methods)}')
html_f_name = f'{results_path_main}/HTML_{exp_type}_{DS_name}_{background_type}_combined.html'

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_type}, background_type: {background_type}] - No of Images:{len(file_success)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth,_,_ in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')
        
# for fff_id,ffp in enumerate(tqdm(files_set)):
for fff_id,file_n in enumerate(tqdm(files)):
    # file_n = ffp.split('.')[0].split('\\')[-1].split('//')[-1].split('_')[0]
    
    # print(file_n)
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    PP = df[df.image==file_n].f_S.iloc[0]
    PL = df[df.image==file_n].f_S

    for ecc_id,ecc in zip(explained_classes_id,explained_classes):
        img_path_heat = f'{results_path}//{file_n}_{ecc}_{background_type}_heatmaps.png'
        img_path_iou = f'{results_path}//{file_n}_{ecc}_{background_type}_iou.png'
        if os.path.exists(img_path_heat):
            with open(html_f_name, 'a') as f:
                f.write(f'<tr><td colspan="{len(methods)+2}">\
                        <img src={img_path_heat} width="99.9%"> <br>\
                        <img src={img_path_iou} width="99.9%"> <br>\
                            <p   style="font-size:22">| {file_n+1:<10}  | {ecc:<10} |  {file_n:<10}  |  {PP:0.5}  |  \
                            </p>\
                        </td>\
                        ')
            with open(html_f_name, 'a') as f:
                    f.write(f'\
                        <tr><td">\
                          ')
            df11 = df[df.intrested_class==ecc_id]
            for sm in auc_methods:
                df1 = df11[(df11.image==file_n)].eval(f'{sm}')
                with open(html_f_name, 'a') as f:
                    f.write(f'\
                            <tr><td>\
                            <div class="collapsible-container">\
                              <div class="collapsible-header">\
                                <span class="toggle-icon"></span>\
                              </div>\
                              <div class="collapsible-content">\
                                <p>\
                                <b>{sm}</b>\
                                </div>\
                            </div>\
                            </td>\
                            ')
                for meth_id,meth in enumerate(methods):
                    df2 = df1.iloc[meth_id]
                    with open(html_f_name, 'a') as f:
                        f.write(f'\
                                <td> \
                                <div class="collapsible-container">\
                                  <div class="collapsible-header">\
                                    <span class="toggle-icon"></span>\
                                  </div>\
                                  <div class="collapsible-content">\
                                    <p>\
                                    {df2:0.5} </p>\
                                    </div>\
                                </div>\
                                </td>\
                                ')
            with open(html_f_name, 'a') as f:
                f.write(f'\
                </tr>\
                ')
        with open(html_f_name, 'a') as f:
            f.write('''
                        </div>
                    </div>
                    </td></tr>
                    ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

# END